In [1]:
import numpy as np
import tensorflow as tf


print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.3.2


In [2]:
dataset = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/dataset.npy")
classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/bin_classes.npy")

In [3]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [4]:
def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [8]:
def split_dataset_and_normalize(perc=0.8):
    shuffled_dataset, shuffled_labels = unison_shuffled_copies(dataset,classes)
    split_point = int(len(dataset)*perc)
    train_ds = shuffled_dataset[:split_point]
    train_labels = shuffled_labels[:split_point]
    val_ds = shuffled_dataset[split_point:]
    val_labels = shuffled_labels[split_point:]
    std_dev = np.std(train_ds)
    mean = np.mean(train_ds)
    maximum = np.max(train_ds)
    min = np.min(train_ds)
    scale = max(maximum, abs(min))
    train_ds = normalize(train_ds, std_dev, mean, scale)
    val_ds = normalize(val_ds, std_dev, mean, scale)
    return train_ds, train_labels, val_ds, val_labels

train_ds, train_labels, val_ds, val_labels = split_dataset_and_normalize()

In [9]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(16,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Bidirectional(LSTM(512, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00001),
                                 recurrent_regularizer=regularizers.l2(0.00001),
                                 bias_regularizer=regularizers.l2(0.00001),
                             recurrent_dropout=0.3)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1024)              69750784  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'],
)

In [11]:
EPOCHS = 500
history = model.fit(
    x= train_ds,
    y = train_labels,
    validation_data=(val_ds, val_labels),
    batch_size=16,
    epochs=EPOCHS
)

Epoch 1/500
38/38 [==============================] - 7s 189ms/step - loss: 0.7805 - accuracy: 0.5211 - val_loss: 0.7797 - val_accuracy: 0.5503

KeyboardInterrupt: 

In [28]:
del model

In [12]:
model.save("audio_model_bidirectional_LSTM.h5")